In [88]:
!pip install geojsoncontour

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [89]:
import pandas as pd
import numpy as np
from datetime import datetime
import time


from tqdm import tqdm

# from scipy.interpolate import griddata
from scipy.interpolate import Rbf

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import folium
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.dates as mdates
import geojsoncontour

from datetime import datetime

from sklearn.impute import SimpleImputer

import ipywidgets as widgets
from ipywidgets import Layout

import warnings
warnings.filterwarnings('ignore')

In [90]:
dataset = pd.read_csv("File1.csv")

In [91]:
# sensors = [[],[],[]]

# for i in range(0, dataset.shape[0]):
#     list = [dataset.iloc[i, 20], dataset.iloc[i, 21], dataset.iloc[i, 22]]
#     if sensors[0].count(list[0])==0 and sensors[1].count(list[1])==0 :
#         sensors[0].append(list[0])
#         sensors[1].append(list[1])
#         sensors[2].append(list[2])


In [92]:
# sensors_map=folium.Map()
# for i in range(0,len(sensors[0])):
#    folium.Marker(
#       location=[sensors[0][i], sensors[1][i]], 
#                 popup=sensors[2][i],
#                  ).add_to(sensors_map)
# sensors_map

In [93]:
print(dataset.isnull().sum())

Unnamed: 0                            0
airQualityLevel                       0
observationDateTime                   0
deviceStatus                          0
aqiMajorPollutant                     0
id                                    0
airQualityIndex                       0
uv.avgOverTime                     5321
o3.avgOverTime                     3240
pm2p5.avgOverTime                  6023
co2.avgOverTime                    4840
pm10.avgOverTime                   5959
co.avgOverTime                     3232
no2.avgOverTime                    3214
airTemperature.avgOverTime         6136
illuminance.avgOverTime            6129
ambientNoise.avgOverTime           6081
so2.avgOverTime                    3252
relativeHumidity.avgOverTime       6142
atmosphericPressure.avgOverTime    6069
latitude                              0
longitude                             0
label                                 0
dtype: int64


#Missing Values

In [94]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
dataset.iloc[:, 6:22] = imputer.fit_transform(dataset.iloc[:, 6:22])

In [95]:
# print(dataset)

In [96]:
print(dataset.isnull().sum())

Unnamed: 0                         0
airQualityLevel                    0
observationDateTime                0
deviceStatus                       0
aqiMajorPollutant                  0
id                                 0
airQualityIndex                    0
uv.avgOverTime                     0
o3.avgOverTime                     0
pm2p5.avgOverTime                  0
co2.avgOverTime                    0
pm10.avgOverTime                   0
co.avgOverTime                     0
no2.avgOverTime                    0
airTemperature.avgOverTime         0
illuminance.avgOverTime            0
ambientNoise.avgOverTime           0
so2.avgOverTime                    0
relativeHumidity.avgOverTime       0
atmosphericPressure.avgOverTime    0
latitude                           0
longitude                          0
label                              0
dtype: int64


In [97]:
for i in range(0, dataset.shape[0]):
    dataset.iloc[i, 2] = dataset.iloc[i , 2][:dataset.iloc[i , 2].find('+')-3]

In [98]:
# print(dataset)

In [99]:
column_choice=['airQualityIndex','airTemperature','ambientNoise','so2','uv','co','co2','atmosphericPressure',
               'illuminance','no2','o3','pm10','pm2p5','relativeHumidity']

In [107]:

prompt1=widgets.HTML(value="")
prompt2=widgets.HTML(value="")
gif_address = 'https://www.uttf.com.ua/assets/images/loader2.gif'

# drop down widget to choose pollutant/property
select_col=widgets.Dropdown(
    options=column_choice,
    value='airQualityIndex',
    description='Property:',
    disabled=False,
)
time_range=[]
for i in range(0, dataset.shape[0]):
    time_range.append(datetime.strptime(dataset.iloc[i , 2], '%Y-%m-%d %H:%M'))
options = [time.strftime('%d %b %H:%M') for time in time_range]
index = (0, len(options)-1)

# Slider to choose time
select_time=widgets.SelectionSlider(
    options=options,
    value=options[100],
    description='Time:',
    disabled=False,
    orientation='horizontal',
    continuous_update=False,
    readout=True,
    width=6000,
)
mywidgets=[select_col]
ui=widgets.VBox([prompt1,select_col,select_time,prompt2],width=2000)

In [108]:
# FUNCTION TO PLOT CONTOUR MAP AFTER CUBIC INTERPOLATION
def pollutant_contour(pollutant,time):
    prompt2.value='Building Contour ...'
    
    # GATHER THE REQUIRED DATA FOT THE CHOSEN TIME
    time=datetime.strptime(time+' 2022','%d %b %H:%M %Y')

    # aqm_latest=aqm[aqm.observationDateTime<time.strftime('%d-%m-%Y %H:%M:%S')]
    # aqm_latest=aqm_latest.sort_values(by='observationDateTime').drop_duplicates(['id'],keep='last')
    # print([dataset.observationDateTime<time.strftime('%d-%m-%Y %H:%M:%S'))
    
    time = str(time)
    time = time[:-3]
    
    latest = dataset.loc[dataset['observationDateTime'] == time]
    latest = latest.sort_values(by='observationDateTime').drop_duplicates(['id'],keep='last')

    if(latest.shape == 0):
        print("Missing Values")
    
    else:
        # print(latest)

        if pollutant != 'airQualityIndex':
            pollutant = pollutant+'.avgOverTime'
        
        # data=aqm_latest.dropna(subset=[pollutant])
        data = latest.dropna(subset=[pollutant])
        readings=data[pollutant]
        x=data['longitude']
        y=data['latitude']
        sensors=list(np.unique(data.id))
    
        #BUILDING A MESH GRID FOR A GIVEN RESOLUTION
        res=1000
        y_arr = np.linspace(np.min(y), np.max(y), res)
        x_arr = np.linspace(np.min(x), np.max(x), res)
        x_mesh, y_mesh = np.meshgrid(x_arr, y_arr)
        
        # PERFORM CUBIC INTERPOLATION
        # interpolated_readings = griddata((x, y), readings, (x_mesh, y_mesh), method='cubic')
        
        rbfi = Rbf(x, y, readings, function='gaussian', mode='1-D')
        interpolated_readings = rbfi(x_mesh, y_mesh)
        
        # PLOT CONTOUR
        levels = 50
        contourf=plt.contourf(x_mesh, y_mesh, interpolated_readings, levels, alpha=0.5, 
                            cmap=cm.jet, linestyles='None', vmin=0)
        plt.close()

        # CONVERT CONTOUR TO GEOJSON REPRESENTION
        geojson = geojsoncontour.contourf_to_geojson(
            contourf=contourf,
            min_angle_deg=3.0,
            ndigits=5,
            stroke_width=1,
            fill_opacity=0.5)

        sensors_map=folium.Map(tiles="cartodbpositron")
        
        # UNCOMMENT THE FOLLOWING LINES TO PLOT THE SENSOR LOCATIONS AS WELL
        # for rsc_id, rsc in rsc_dict.items():
        #     if rsc_id in sensors:
        #         folium.Marker([rsc["coordinates"][1], rsc["coordinates"][0]],
        #                       tooltip=rsc['label']).add_to(sensors_map)

        # PLOT CONTOUR ON MAP WITH FOLIUM 
        folium.GeoJson(
            geojson,
            style_function=lambda x: {
                'color':     x['properties']['stroke'],
                'weight':    x['properties']['stroke-width'],
                'fillColor': x['properties']['fill'],
                'opacity':   0.6,
            }).add_to(sensors_map)

        sensors_map.fit_bounds(sensors_map.get_bounds())
        
        prompt2.value=str(time)
        
        display(sensors_map)

In [109]:

display(ui)
widgets.interactive_output(pollutant_contour,{'pollutant':select_col,'time':select_time})
      


Output()

In [103]:
print(dataset.loc[dataset['observationDateTime'] == '2022-01-22 17:01'])

       Unnamed: 0 airQualityLevel observationDateTime deviceStatus  \
78188       81615    SATISFACTORY    2022-01-22 17:01       ACTIVE   
78189       81616    SATISFACTORY    2022-01-22 17:01       ACTIVE   
78190       81617        MODERATE    2022-01-22 17:01       ACTIVE   
78191       81618        MODERATE    2022-01-22 17:01       ACTIVE   
78192       81619    SATISFACTORY    2022-01-22 17:01       ACTIVE   
78193       81620        MODERATE    2022-01-22 17:01       ACTIVE   
78194       81621    SATISFACTORY    2022-01-22 17:01       ACTIVE   
78195       81622    SATISFACTORY    2022-01-22 17:01       ACTIVE   
78196       81623    SATISFACTORY    2022-01-22 17:01       ACTIVE   
78197       81624    SATISFACTORY    2022-01-22 17:01       ACTIVE   
78198       81625        MODERATE    2022-01-22 17:01       ACTIVE   
78199       81626            GOOD    2022-01-22 17:01       ACTIVE   
78200       81627    SATISFACTORY    2022-01-22 17:01       ACTIVE   
78201       81628   